In [1]:
from ast import literal_eval
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

all_stopwords = set(stopwords.words('english') + list(string.punctuation))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/temilajumoke/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/temilajumoke/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data Preprocessing

In [3]:
CONTENT_ROOT_PATH = "../Entity_Recognition/Neighborhood_Separated_Articles"

In [4]:
data_file_name_dict = {
    '2014':'2014.csv',
    '2015':'2015.csv',
    '2016':'2016.csv',
    '2017':'2017.csv',
    '2018':'2018.csv',
}

In [5]:
file_name = data_file_name_dict['2014']
df = pd.read_csv(f'{CONTENT_ROOT_PATH}/{file_name}')
df = df.replace(to_replace ="('no article', 'no_id')", value="")
df.head(5)

,Unnamed: 0,dorchester,roxbury,mattapan,hyde_park,fenway,beacon_hill,downtown,south_boston,east_boston,...,charlestown,brighton,allston,west_end,roslindale,north_end,mission_hill,harbor_islands,longwood_medical_area,west_roxbury
0,0,('Before entering a South African prison in 19...,('n individuals who worked in finance insuran...,('The public scrap over who will serve as the ...,('The Boston Police Department has issued a co...,('Wahlburgers the burger joint owned by Dorch...,('Martin J Walsh becomes the 48th mayor of Bo...,('Last week Mayor Marty Walsh declared to the...,('The 20 year old who allegedly beat a disable...,('Saying that Massachusetts is facing “the big...,...,('Bright skies belie the early spring chill as...,('Boston’s new mayor Martin J Walsh has rep...,('A small army of city inspectors will soon be...,('Shakespeare’s “A Midsummer Night’s Dream” se...,('Last February Brandon John joined more than...,('When the offered season ticket holders the ...,('They were immigrants from Iraq and India th...,('Rain crashes down on onto the tarp thunder ...,,('It’s been almost a full month since Thomas M...
1,1,('A woman’s toe was bitten off during a fight ...,('Dr Kenneth C Edelin whose historic 1975 m...,('In what police described as a horrific trage...,('nOn the weekend before he was to be sworn in...,('n n n and are expanding their burger bus...,('One bone chilling Saturday night about 11 mo...,('The Federal Reserve must take a “patient app...,('n and Michelle Wu stood side by side at the ...,('MEMPHIS — The thank yous were almost endless...,...,('As neighbors living a world apart they awok...,('The 26 year old Chinese entrepreneur had jus...,('n n for help The harrowing leaps from smoky...,('nis here the nascent organizing committee...,('Don’t ask how the sausage is made the old s...,('As America’s most Irish American city it co...,('Beth Israel Deaconess Medical Center said We...,,,('Mayor Martin J Walsh’s pick for acting fire...
2,2,('It’s the ideal time for a tech journalist to...,('When they go up they form the centerpiece o...,('The fatal shooting today of a 9 year old Mat...,('Boston saw its first two homicides of 2014 o...,('Mayor Martin J Walsh plans to set aside spa...,('nhe Boston Public Library is using three dim...,('ncooler buses For decades economists like ...,('Jack Conley who grew up in South Boston and...,('Marty Walsh has turned the casino licensing ...,...,('When Boston firefighters pulled up to a vaca...,('It was just some children playing baseball o...,('n n for help The harrowing leaps from smoky...,('nis here the nascent organizing committee...,('Former Boston councilor Felix D Arroyo the...,('In nine days Wegmans will open a new superm...,('The Rev Joseph Hung Duc Tran had been ordai...,,,('His cousin would come to Children’s Hospital...
3,3,('The first major snowstorm of 2014 has intens...,('Good Morning Chief Justice Ireland Presiden...,('During a highly publicized Boston Police Dep...,('An elderly woman became Boston’s first fire ...,('By last Christmas Frank Nunes seemed to hav...,('QUINCY — Massachusetts Democrats saw a new B...,('When Ari S Heckman was growing up in Provid...,('Mayor Martin J Walsh said Wednesday that he...,('Marty Walsh has turned the casino licensing ...,...,('Two Charlestown teenagers — a 14 year old gi...,('Boston police are asking for the public’s he...,('n n for help The harrowing leaps from smoky...,('The first thing everyone noticed about Dawnn...,('nn the middle of evening rush hour at South ...,('n n n nThe 150 dancing singing and drummin...,('nLittle about the ordinary looking two famil...,,,('The is offering 15 percent off purchases fr...
4,4,('One Fund Boston which initially collected $...,('Good Morning Chief Justice Ireland Presiden...,('At Greater Love Tabernacle Church today mor...,('n officials in Boston want to give 4 500 mid...,('SALEM — Martha Coakley had already made the ...,('The state’s highest court has thrown out the...,('Marty Walsh had never met 

In [6]:
def parse_value(x):
    try:
        return literal_eval(x)   
    except Exception as e:
        return None

def parse_column_values(data_frame):
    for i in range(1, len(data_frame.columns)):
        column_name = data_frame.columns[i]
        data_frame[column_name] = data_frame[column_name].apply(lambda x: parse_value(x))   
    return data_frame.iloc[: , 1:]

In [7]:
def tokenize(text):
    if text is None:
        return None

    text = text[0]
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text.lower())
    tokens = []
    for word in nltk.word_tokenize(text):
        if word in all_stopwords:
            continue
        tokens.append(word)

    return " ".join(tokens)

In [8]:
def generate_training_test_doc_set(data_frame, train_column):
    training_values = []
    test_values = []
    for i in range(len(data_frame.columns)):
        column_name = data_frame.columns[i]
        if (column_name == train_column):
            train_val = data_frame[column_name].apply(lambda x: tokenize(x))
            train_val = list(filter(None.__ne__, train_val))
            training_values.extend(train_val)
        else:
            test_val = data_frame[column_name].apply(lambda x: tokenize(x))
            test_val = list(filter(None.__ne__, test_val))
            test_values.extend(test_val)

    return (training_values, test_values)

In [9]:
def generate_frequency_matrix(year, file_name):
  
    important_neighborhood_words = {}

    df = pd.read_csv(f'{CONTENT_ROOT_PATH}/{file_name}')
    parsed_df = parse_column_values(df)

    print(f"generating tf-idf matrix for {year} dataset")
  
    for i in range(len(parsed_df.columns)):
        column_name = parsed_df.columns[i]
        print(f"generating tf-idf for {column_name} - {year}")
        
        (training_values, test_values) = generate_training_test_doc_set(parsed_df, column_name)

        if len(training_values) == 0 or len(test_values) == 0:
            continue

        tfidf_vectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english', use_idf=True)

        tfidf_vectorizer.fit(test_values)
        tfidf_term_vectors  = tfidf_vectorizer.transform(training_values)

        feature_names = np.array(tfidf_vectorizer.get_feature_names())
        
        df = pd.DataFrame(tfidf_vectorizer.idf_, index = feature_names, columns=["weights"])
        df = df.sort_values(by=['weights'], ascending=False)
        print(df.head(10))
        
        df.to_csv(f'../data/processed/TF-IDF/{year}/tf-idf-{column_name}.csv')
    
    return (tfidf_term_vectors, feature_names)

In [10]:
top_neighborhood_words = generate_frequency_matrix(2014, data_file_name_dict['2014'])

generating tf-idf matrix for 2014 dataset
generating tf-idf for dorchester - 2014
            weights
laypeople  8.015263
madsen     8.015263
lynnel     8.015263
lyon       8.015263
lyonne     8.015263
lyra       8.015263
lysko      8.015263
macafee    8.015263
macaluso   8.015263
macaroni   8.015263
generating tf-idf for roxbury - 2014
            weights
ﬁrearms    8.134492
loitering  8.134492
londergan  8.134492
lomp       8.134492
lombos     8.134492
lombard    8.134492
lomax      8.134492
lol        8.134492
loiter     8.134492
lyra       8.134492
generating tf-idf for mattapan - 2014
               weights
evarts        8.180451
gastropub     8.180451
robbinsville  8.180451
gatekeeper    8.180451
robb          8.180451
roasting      8.180451
roasters      8.180451
roaster       8.180451
gatlin        8.180451
gatsby        8.180451
generating tf-idf for hyde_park - 2014
             weights
ﬁrearms     8.177401
irrigation  8.177401
todisco     8.177401
iraqi       8.177401
toddli

In [11]:
top_neighborhood_words = generate_frequency_matrix(2015, data_file_name_dict['2015'])

generating tf-idf matrix for 2015 dataset
generating tf-idf for dorchester - 2015
                weights
00             8.111512
ofseriouseats  8.111512
ojikutu        8.111512
ojala          8.111512
oils           8.111512
oiler          8.111512
oil3           8.111512
ohta           8.111512
ohrenberger    8.111512
ohioan         8.111512
generating tf-idf for roxbury - 2015
                weights
00             8.257355
nwake          8.257355
editions       8.257355
nwatson        8.257355
editorialists  8.257355
nwater         8.257355
nwatching      8.257355
edlen          8.257355
edmond         8.257355
nwasserman     8.257355
generating tf-idf for mattapan - 2015
            weights
00         8.335634
dzhokar    8.335634
ntogether  8.335634
díaz       8.335634
dürer      8.335634
ntitciana  8.335634
ntime      8.335634
ntido      8.335634
nthursday  8.335634
eain       8.335634
generating tf-idf for hyde_park - 2015
                weights
00             8.320527
loring  

In [12]:
top_neighborhood_words = generate_frequency_matrix(2016, data_file_name_dict['2016'])

generating tf-idf matrix for 2016 dataset
generating tf-idf for dorchester - 2016
               weights
único         8.144802
sherdrick     8.144802
genders       8.144802
generaciones  8.144802
shied         8.144802
generals      8.144802
shhhh         8.144802
sheyla        8.144802
sheryll       8.144802
sherrod       8.144802
generating tf-idf for roxbury - 2016
                weights
único          8.288586
reindeer       8.288586
fannie         8.288586
fanning        8.288586
reinspect      8.288586
fanta          8.288586
reinier        8.288586
fantastically  8.288586
reinharz       8.288586
fanáticos      8.288586
generating tf-idf for mattapan - 2016
              weights
único        8.389564
galveston    8.389564
galway       8.389564
gamblers     8.389564
sequestered  8.389564
sequencing   8.389564
sequenced    8.389564
gammons      8.389564
ganado       8.389564
gandhi       8.389564
generating tf-idf for hyde_park - 2016
               weights
único         8.389564

In [13]:
top_neighborhood_words = generate_frequency_matrix(2017, data_file_name_dict['2017'])

generating tf-idf matrix for 2017 dataset
generating tf-idf for dorchester - 2017
               weights
parapet       8.246368
ncomcast      8.246368
ndracut       8.246368
ndrowning     8.246368
ndue          8.246368
nduggie       8.246368
nduties       8.246368
cog           8.246368
cofounders    8.246368
neanderthals  8.246368
generating tf-idf for roxbury - 2017
             weights
handlers    8.451242
schmear     8.451242
mcmurphy    8.451242
mcmillan    8.451242
schaphorst  8.451242
schaub      8.451242
entrust     8.451242
schemes     8.451242
schiappa    8.451242
schifone    8.451242
generating tf-idf for mattapan - 2017
               weights
00            8.552762
ndemetrius    8.552762
ndistrict     8.552762
ndispiriting  8.552762
ndimitris     8.552762
ndennis       8.552762
cocozza       8.552762
ndempsey      8.552762
ndefense      8.552762
nec           8.552762
generating tf-idf for hyde_park - 2017
             weights
programme   8.552762
mcgaughan   8.552762
scat

In [14]:
top_neighborhood_words = generate_frequency_matrix(2018, data_file_name_dict['2018'])

generating tf-idf matrix for 2018 dataset
generating tf-idf for dorchester - 2018
               weights
guerra        8.621195
pmfitchburg   8.621195
pmfranklin    8.621195
legitimizing  8.621195
mohawk        8.621195
thurman       8.621195
cohost        8.621195
legatum       8.621195
pmlexington   8.621195
cohosts       8.621195
generating tf-idf for roxbury - 2018
                weights
quietest       8.831022
semifreddo     8.831022
halpin         8.831022
semicircles    8.831022
macomber       8.831022
anniversity    8.831022
chunsu         8.831022
urges          8.831022
madagascar     8.831022
disfigurement  8.831022
generating tf-idf for mattapan - 2018
                   weights
newlyweds         8.944492
schoolsdedham     8.944492
schoolslowell     8.944492
schoolslynn       8.944492
aholiday          8.944492
schoolslynnfield  8.944492
lb                8.944492
quarantine        8.944492
schoolsmalden     8.944492
crosschecks       8.944492
generating tf-idf for hyde_pa